In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta.tables import *
from delta import *
from pyspark.sql.functions import *
import psycopg2
import pandas as pd
import os

In [3]:
env_vars = !cat Script/.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [4]:
builder = pyspark.sql.SparkSession.builder.appName("PySpark_Postgres_test").config("spark.jars", os.environ.get('SPARKJAR')).config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").master("local").appName("PySpark_Postgres_test")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/16 17:30:27 WARN Utils: Your hostname, MacBook-Pro-de-Renato.local resolves to a loopback address: 127.0.0.1; using 10.0.0.102 instead (on interface en0)
22/05/16 17:30:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/renabouj/.ivy2/cache
The jars for the packages stored in: /Users/renabouj/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5cfb46ba-4456-485b-a71f-ed3987b3e1a9;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 183ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnl

In [5]:
dataf = spark.read.format("jdbc").option("url", os.environ.get('DB_CONNECTION_STRING')) \
    .option("driver", "org.postgresql.Driver").option("dbtable", "sales2") \
    .option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')).load()

In [6]:
dataf.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: integer (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: integer (nullable = true)
 |-- Unit_Cost: integer (nullable = true)
 |-- Total_Revenue: integer (nullable = true)
 |-- Total_Cost: integer (nullable = true)
 |-- Total_Profit: integer (nullable = true)



In [7]:
dataf.count()

254160

In [9]:
dataf.where(dataf.Order_ID == 451691138).show()
deltadf = spark.read.format("delta").load("/tmp/output/bancodeucerto2")
deltadf.where(deltadf.Order_ID == 451691138).show()

+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|  Country|Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|Australia|     Meat|       Online|             C|  4/4/2011|451691138|5/23/2011|      3300|       500|      364|      1814127|   1568167|      245960|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+



+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|  Country|Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|Australia|     Meat|       Online|             C|  4/4/2011|451691138|5/23/2011|      3300|       500|      364|      1814127|   1568167|      245960|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+



In [10]:
teste = DeltaTable.forPath(spark, "/tmp/output/bancodeucerto2")

In [11]:
teste.update(
  condition = expr("Order_ID == 451691138"),
  set = { "Unit_Cost": expr("Unit_Cost + 79") })

teste.update(
  condition = expr("Order_ID == 451691138"),
  set = { "Unit_Price": expr("Unit_Price - 79") })

teste.update(
  condition = expr("Order_ID == 451691138"),
  set = { "Units_Sold": expr("Units_Sold + 1000") })

In [12]:
df_final = spark.read.format("delta").load("/tmp/output/bancodeucerto2")
df_final.where(df_final.Order_ID == 451691138).show()

+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|  Country|Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|Australia|     Meat|       Online|             C|  4/4/2011|451691138|5/23/2011|      4300|       421|      443|      1814127|   1568167|      245960|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+



In [14]:
df_final.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", os.environ.get('DB_CONNECTION_STRING')) \
    .option("dbtable", "sales2").option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')) \
    .mode("overwrite").save()

In [ ]:
dataf.write.format("delta").save("/tmp/output/bancodeucerto2")

In [7]:
dataf.where(dataf.Order_ID == 451691138).withColumn("Unit_Price", (pyspark.sql.functions.col("Unit_Price") + 79))
dataf.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", "jdbc:postgresql://localhost:5432/tendencias") \
    .option("dbtable", "sales1").option("user", "root").option("password", "secret") \
    .mode("overwrite").save()

In [8]:
dataf.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", "jdbc:postgresql://localhost:5432/tendencias") \
    .option("dbtable", "sales1").option("user", "root").option("password", "secret") \
    .mode("overwrite").save()

In [10]:
dataf.write.format("parquet").save("/tmp/output/bancodeucerto")

In [7]:
parDF = spark.read.parquet("/tmp/output/bancodeucerto")

In [8]:
dataf.write.format("delta").save("/tmp/output/bancodeucerto2")

In [18]:
deltadf = spark.read.format("delta").load("/tmp/output/bancodeucerto2")

In [19]:
deltadf.where(deltadf.id == 254159).show()

+------+------+---------+---------------+-------------+--------------+-------------------+---------+-------------------+----------+----------+---------+-------------+----------+------------+
|    id|Region|  Country|      Item_Type|Sales_Channel|Order_Priority|         Order_Date| Order_ID|          Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+------+------+---------+---------------+-------------+--------------+-------------------+---------+-------------------+----------+----------+---------+-------------+----------+------------+
|254159|  Asia|Singapore|Office Supplies|       Online|             C|2010-06-06 21:00:00|953051793|2010-07-18 21:00:00|      8059|       651|      600|      5248101|   4230652|     1017448|
+------+------+---------+---------------+-------------+--------------+-------------------+---------+-------------------+----------+----------+---------+-------------+----------+------------+



In [20]:
deltadf.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", "jdbc:postgresql://localhost:5432/tendencias").option("dbtable", "sales2").option("user", "root").option("password", "secret").mode("overwrite").save()

22/05/16 13:47:57 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 975219 ms exceeds timeout 120000 ms
22/05/16 13:47:57 WARN SparkContext: Killing executors is not supported by current scheduler.


In [9]:
teste = DeltaTable.forPath(spark, "/tmp/output/bancodeucerto2")

In [13]:
teste.update(
  condition = expr("Order_ID == 451691138"),
  set = { "Unit_Cost": expr("Unit_Cost - 79") })

teste.update(
  condition = expr("Order_ID == 451691138"),
  set = { "Unit_Price": expr("Unit_Price + 79") })

teste.update(
  condition = expr("Order_ID == 451691138"),
  set = { "Units_Sold": expr("Units_Sold - 1000") })

In [14]:
deltadf = spark.read.format("delta").load("/tmp/output/bancodeucerto2")

In [15]:
deltadf.where(deltadf.Order_ID == 451691138).show()

+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|  Country|Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|Australia|     Meat|       Online|             C|  4/4/2011|451691138|5/23/2011|      3300|       500|      364|      1814127|   1568167|      245960|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+



In [16]:
deltadf.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", "jdbc:postgresql://localhost:5432/tendencias").option("dbtable", "sales2").option("user", "root").option("password", "secret").mode("overwrite").save()